In [3]:
!git clone https://github.com/zach401/acnportal.git
!pip install acnportal/.

fatal: destination path 'acnportal' already exists and is not an empty directory.


Processing c:\users\omi\documents\uni\diplomovka\github\code rep\diplomovka_code\acnportal\my_experiments\python_implementation\mpc\acnportal
Using legacy 'setup.py install' for acnportal, since package 'wheel' is not installed.
  Attempting uninstall: acnportal
    Found existing installation: acnportal 0.3.3
    Uninstalling acnportal-0.3.3:
      Successfully uninstalled acnportal-0.3.3
    Running setup.py install for acnportal: started
    Running setup.py install for acnportal: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
You should consider upgrading via the 'C:\Users\OMI\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from copy import deepcopy

from acnportal import algorithms
from acnportal import acnsim
from mpc_sched import *
from mpc_optim import *


ModuleNotFoundError: No module named 'cvxpy'

In [ ]:




# -- Experiment Parameters ---------------------------------------------------------------------------------------------
timezone = pytz.timezone('America/Los_Angeles')
start = timezone.localize(datetime(2018, 9, 5))
end = timezone.localize(datetime(2018, 9, 6))
period = 5  # minute
voltage = 220  # volts
default_battery_power = 32 * voltage / 1000 # kW
site = 'caltech'

# -- Network -----------------------------------------------------------------------------------------------------------
cn = acnsim.sites.caltech_acn(basic_evse=True, voltage=voltage)

# -- Events ------------------------------------------------------------------------------------------------------------
API_KEY = 'DEMO_TOKEN'
events = acnsim.acndata_events.generate_events(API_KEY, site, start, end, period, voltage, default_battery_power)


# -- Scheduling Algorithm ----------------------------------------------------------------------------------------------
# sch = EarliestDeadlineFirstAlgo(increment=1)
sch2 = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

# objectives can be:
#  maximise energy
#  minimize costs/ maximise profit
# charge as quickly as possible
# smoothen charging rates
obj = Objective()
objective = [
    obj.maximize_charging_energy(obj),
    obj.minimize_customers_chaging_costs(obj),
    obj.charge_as_quickly_as_possible(self=obj),
    # smoothen_charging_rates
]
sch_mpc = MyMpcAlgorithm(objective=objective)


sim_mpc = acnsim.Simulator(deepcopy(cn), sch2, deepcopy(events),start, period=period, verbose=False)
sim_mpc.run()
a = 0

In [ ]:
# For comparison we will also run the builtin earliest deadline first algorithm
# sim2 = acnsim.Simulator(deepcopy(cn), sch2, deepcopy(events), start, period=period, verbose=False)
# sim2.run()